<a href="https://colab.research.google.com/github/jaksonvelasquez/cun/blob/main/Extraer_tablas_de_un_pdf_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Versión final con PDFPLumber
import sys
import subprocess

# Función para instalar una librería y ocultar la salida del proceso de instalación
def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package], stdout=subprocess.DEVNULL)
        __import__(package)

# Instalar PDFPlumber si no está disponible
install_and_import('pdfplumber')

import pdfplumber
import pandas as pd
from google.colab import files
from IPython.display import display

def extract_tables_from_pdf(pdf_path, pages):
    all_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        # Extraer tablas solo de las páginas especificadas
        for page_number in pages:
            page = pdf.pages[page_number - 1]  # Ajuste de índice de página
            page_tables = page.extract_tables()
            for table in page_tables:
                if table:
                    # Convertir la tabla a un DataFrame y añadirlo a la lista
                    df = pd.DataFrame(table[1:], columns=table[0])
                    all_tables.append(df)
    return all_tables

def tables_to_excel(tables, indices, excel_path):
    # Crear un archivo Excel solo con las tablas seleccionadas
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for index in indices:
            tables[index].to_excel(writer, sheet_name=f'Tabla {index + 1}', index=False)

# Cargar el archivo PDF
uploaded = files.upload()
pdf_path = next(iter(uploaded.keys()))

# Solicitar al usuario las páginas donde se encuentran las tablas
page_numbers = input("Ingresa los números de las páginas donde están las tablas, separados por comas (e.g., 1,3,5): ")
page_numbers = [int(page.strip()) for page in page_numbers.split(',')]

# Extraer las tablas del PDF en las páginas especificadas
tables = extract_tables_from_pdf(pdf_path, page_numbers)

# Mostrar las tablas al usuario y pedir que seleccione las que desea guardar
print("Tablas extraídas:")
for i, table in enumerate(tables):
    print(f"\nTabla {i + 1}:")
    display(table)  # Mejor visualización usando display

selected_indices = input("Ingresa los números de las tablas que deseas guardar, separados por comas (e.g., 1,3,5): ")
selected_indices = [int(index.strip()) - 1 for index in selected_indices.split(',')]

# Ruta del archivo Excel de salida
excel_path = 'tablas_extraidas.xlsx'

# Convertir las tablas seleccionadas a un archivo Excel
tables_to_excel(tables, selected_indices, excel_path)

# Descargar el archivo Excel
files.download(excel_path)